In [1]:
import json
import pandas as pd
import time
import gcsfs

In [2]:
from google.cloud import storage
from google.cloud import bigquery
from pathlib import Path
import os

In [3]:
PROJ_ROOT = Path().resolve().parent
KEYS_DIR = PROJ_ROOT / 'keys' 
keys = KEYS_DIR / 'keys-for-smart-science-muse-data.json'

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(keys)

In [5]:
project = 'smart-science'
dataset_id = 'Learners_Questions'
table_id = 'Combined_tables_39k'
bucket_name = 'muse-data'

# Setup Storage vars
storage_client = storage.Client(project=project)
bucket = storage_client.get_bucket('muse-data')

# Setup BigQuery vars
bq_client = bigquery.Client()
dataset_ref = bq_client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)

#destination
destination_uri = "gs://{}/{}".format(bucket_name, "combined_tables_raw.json")

In [7]:
#set job configuration to extract job as JSON files
job_config = bigquery.ExtractJobConfig()
job_config.destination_format = 'NEWLINE_DELIMITED_JSON'
job_config.write_disposition = 'WRITE_TRUNCATE'

In [8]:
extract_job = bq_client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
    job_config=job_config
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)

Exported smart-science:Learners_Questions.Combined_Tables to gs://muse-data/combined_tables_raw.json


In [9]:
df = pd.read_json(destination_uri, lines=True)

In [10]:
df.shape

(32687, 11)